Threshold Paillier Cryptosystem
=============

## A Distributed Decryption for Paillier

The protocol offers a distributive decryption for Paillier with simulation based security against malicious adversaries without randomness extraction. It is comprised of the following two subprotocols:

1. The parties produce shares of a value d similarly to the **Damgard-Jurik scheme**

2. The parties run the distributed decryption algorithm using their shares.

use $g = 1 + N$ as a generator of the subgroup of $Z^∗_{N^2}$ of order $N$. Encryption of a plaintext $m$ with randomness $r$ is then,

$$
Enc_N(m, r) = (1+N)^m \cdot r^N \mod N^2
$$

### Generating a Shared Paillier Decryption Key

We now present our protocol for generating a shared Paillier decryption key. As stated, similarly to **Damgard and Jurik** , we share a decryption exponent as follows:



$$
d \equiv \left\{
\begin{aligned}
0 \mod \phi(N) \\
1 \mod N
\end{aligned}
\right\}
$$

#### A distributed generation of a shared Paillier decryption key with passive security:

* **Input**:

A public **RSA** modulus $N=pq$ with unknow factorization, additive shares of $\phi(N)$:

$$
sk_0 = N-p_0-q_0+1\\
sk_1 = -p_1-q_2
$$

$sk_0, sk_1$ hold by $P_0, and P_1$ respectively, A public **ElGamel** key $(g,h)$ with secret key shared between the parities. A public Pailler key $N_0>>N^3$ with the secret key hold by $P_0$

In [1]:
from klefki.numbers.primes import generate_prime
from klefki.numbers import lcm

In [2]:
P, Q = generate_prime(32), generate_prime(32)
N = P * Q
phi_N = lcm(P-1, Q-1)

In [3]:
p0, q0 = generate_prime(8), generate_prime(8)
p1, q1 = generate_prime(8), generate_prime(8)

In [4]:
sk0 = N - p0 - q0 + 1
sk1 = -p1 - q1

In [5]:
from klefki.crypto.elgamal import ElGamal

In [6]:
x = 42
el = ElGamal(x)

In [7]:
g, h = el.pubkey

* **The  protocol**

1. $P_0$ encrypts $sk_0$ using $N_0$ and sends this to $P_1$

In [8]:
from klefki.crypto.paillier import Paillier

In [9]:
p0 = Paillier(generate_prime(32*2), generate_prime(32*2))

In [10]:
p0.E(sk0)

FiniteField::7694557994242821535810815305054920839718274878725359546951458478425122821665

In [11]:
p0.D(p0.E(sk0))

FiniteField::15765931407588335538

In [12]:
sk0

15765931407588335538

----------------

2. $P_1$ picks $r_1 \in Z_N$ and $r_\sigma \in Z_{2^{logN+k}}$ uniformly at random (for a statistical parameter $κ$ that enables to mask the secret key). $P_1$ computes an encryption of

$$
(sk_0 + sk_1) \cdot r_1 + N \cdot r_\sigma
$$

using the homomorphic property of Paillier encryption. This is rerandomized and sent to $P_0$ .

In [13]:
from klefki.types.algebra.utils import randfield
from klefki.types.algebra.meta import field
from math import log

In [25]:
r1 = randfield(field(N)).value
ro = randfield(field(int(2 ** log(N, 2)))).value

In [26]:
r0 = p0.E((sk0+sk1) * r1 + N * ro)

------------



3. $P_0$ decrypts, thus obtaining plaintext $r_0 ; P_0$ computes $r_0^{−1} \mod N$ and encrypts this as well as plaintext $sk_0 (r_0^{−1} \mod N)$ under public-key $N_0$ . Both ciphertexts are sent to $P_1$

In [27]:
r0 = p0.D(r0)

In [28]:
p0.E(sk0 * (~r0).value)

FiniteField::7726238478524501936606961042903863745924174668767727170943505671421095696668

--------------
4. Based on the encryptions of $r_0^{-1} \mod N$ and $sk_0(r_0^{-1} \mod N)$, $P_1$ computes an encryption of:

\begin{align}
d&=(sk_0(r_0^{-1} \mod N) \cdot r_1 + (r_0^{-1} \mod N)(sk_1 \cdot r_1))\\
&=r1(sk_0+sk_1)(r_0^{01} \mod N)\\
&=r_1 \cdot \phi(N) \cdot (r_0^{-1} \mod N)
\end{align}

$P_1$ then picks $\bar{d}_1$ uniformly at random in $Z_{2^3logN+κ}$ , and computes and rerandomizes an encryption of $d + \bar{d}_1$ . This is sent to $P_0$ and ﬁnally, $P_1$ sets its share of d to the integer $-\bar{d}_1$ .

In [40]:
e_d = p0.E(r1 * phi_N * (~r0).value)

In [43]:
d1_ = randfield(field(int(2 ** (3 * log(N, 2))))).value

d0 = p0.E(d1_) + e_d

In [46]:
-d0

FiniteField::13510668490969032060596688711845121177207972889643198493299521200913585996182

----------
5. $P_0$ decrypts and obtains $d_0$ : its share of $d$.

In [47]:
p0.D(d0)

FiniteField::39472400819079321315216871332196193545

Ref:

* Carmit Hazay, Gert Læssøe Mikkelsen, etc.., Efﬁcient RSA Key Generation and Threshold Paillier in the Two-Party Setting